In [ ]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9,hi;q=0.8" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kagglesdsdata/competitions/3362/31148/train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1618475858&Signature=LURs41GoHWC2eCuae6RfmbQb9bq8t5RLyDe9CPB2WAUX7XrfBwWQ2mZBK8si0RLJW6O2hNihGsmxYWFBrZGS4Sv4euIr2GSHA9ZXmEuPo0t3R4gxAaU5PcvXRgcCCkCVREIFBFYtJzlhLGGd8tuXwogIjgdXmkOlY2ntd1Ky9OTqLOUSagzKcQ6fOyU9%2Fbk%2BDZUyWHmOdZ4tN%2F1GuOSXeyP61vW8LE0Eo6UGJrvUMSd1m%2BU75UHzrl3HR%2F0wrnZWGltfUa1r4Tzb6GYplmVn8SQwLEY44UtEFhh2gZTuIbsQs%2FIDQ1MHhDCmGOACFpSwLYGcOlv5Yxk4JYTFbsDYsw%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.zip" -c -O 'train.zip'

In [ ]:
!unzip train.zip

##  cat=0, dog=1 because flow_from_directory numbers categories based on alphabetical order

In [ ]:
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
# create directories
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['dogs/', 'cats/']
	for labldir in labeldirs:
		newdir = dataset_home + subdir + labldir
		makedirs(newdir, exist_ok=True)
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.20
# copy training dataset images into subdirectories
src_directory = 'train/'
for file in listdir(src_directory):
	src = src_directory + '/' + file
	dst_dir = 'train/'
	if random() < val_ratio:
		dst_dir = 'test/'
	if file.startswith('cat'):
		dst = dataset_home + dst_dir + 'cats/'  + file
		copyfile(src, dst)
	elif file.startswith('dog'):
		dst = dataset_home + dst_dir + 'dogs/'  + file
		copyfile(src, dst)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, Activation

from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.utils import to_categorical

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [ ]:
classifier = Sequential()

classifier.add(Conv2D(32, (3, 3), input_shape = (200, 200, 3), activation = 'relu', kernel_initializer='he_uniform'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer='he_uniform'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer='he_uniform'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

opt = SGD(lr=0.001, momentum=0.9)
classifier.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 198, 198, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 46, 46, 128)       73856     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 67712)            

In [ ]:
data_generator = ImageDataGenerator(rescale = 1./255)

train_set = data_generator.flow_from_directory('/content/dataset_dogs_vs_cats/train/',
                                                 target_size = (200, 200),
                                                 batch_size = 64,
                                                 class_mode = 'binary')

val_set = data_generator.flow_from_directory('/content/dataset_dogs_vs_cats/test/',
                                                 target_size = (200, 200),
                                                 batch_size = 64,
                                                 class_mode = 'binary')

Found 19910 images belonging to 2 classes.
Found 5090 images belonging to 2 classes.


In [ ]:
classifier.fit_generator(train_set,
                         epochs = 20,
                         validation_data = val_set,
                         validation_steps=len(val_set),
                         steps_per_epoch=len(train_set))

Epoch 1/20
312/312 [==============================] - 75s 240ms/step - loss: 0.7372 - accuracy: 0.5296 - val_loss: 0.6617 - val_accuracy: 0.6210
Epoch 2/20
312/312 [==============================] - 75s 241ms/step - loss: 0.6625 - accuracy: 0.5956 - val_loss: 0.6547 - val_accuracy: 0.6128
Epoch 3/20
312/312 [==============================] - 74s 238ms/step - loss: 0.6516 - accuracy: 0.6091 - val_loss: 0.6263 - val_accuracy: 0.6460
Epoch 4/20
312/312 [==============================] - 75s 240ms/step - loss: 0.6310 - accuracy: 0.6425 - val_loss: 0.6311 - val_accuracy: 0.6269
Epoch 5/20
312/312 [==============================] - 74s 237ms/step - loss: 0.5965 - accuracy: 0.6752 - val_loss: 0.5634 - val_accuracy: 0.7122
Epoch 6/20
312/312 [==============================] - 74s 237ms/step - loss: 0.5686 - accuracy: 0.7017 - val_loss: 0.5608 - val_accuracy: 0.7045
Epoch 7/20
312/312 [==============================] - 74s 237ms/step - loss: 0.5542 - accuracy: 0.7191 - val_loss: 0.5298 - val_ac

In [ ]:
classifier.save("model_catsVSdogs.h5")

In [ ]:
score = classifier.evaluate_generator(val_set, verbose=0)
print(score[0])
print(score[1])

0.4496488869190216
0.8021610975265503


In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import warnings
warnings.filterwarnings('ignore')
 
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(200, 200))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 200, 200, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img
 
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('/content/dog.4011.jpg')
	# load model
	model = load_model('/content/model_catsVSdogs.h5')
	# predict the class
	result = model.predict(img)
	print(result[0])
 
# entry point, run the example
run_example()

[1.]


# Web Application Using Flask and Flask ngrok

In [ ]:
!pip install flask-ngrok

In [ ]:
import tensorflow as tf
from flask import Flask, render_template, request, send_from_directory
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from tensorflow.keras import backend as K
from flask_ngrok import run_with_ngrok

import warnings
warnings.filterwarnings('ignore')

app = Flask(__name__, template_folder='/content/templates/')
run_with_ngrok(app)

# dir_path = os.path.dirname(os.path.realpath(__file__))
UPLOAD_FOLDER = '/content/uploads/'
MODEL_FOLDER = '/content/model'

def predict(filename):
    img = load_img(filename, target_size=(200, 200))
    img = img_to_array(img)
    img = img.reshape(1, 200, 200, 3)
    img = img.astype('float32')
    img = img - [123.68, 116.779, 103.939]
    model = load_model(MODEL_FOLDER + '/model_catsVSdogs.h5')
    result = model.predict(img)
    print(result)
    result = result.flatten()
    result = round(result[0])
    K.clear_session()
    return result

@app.route('/')
def home():
   return render_template('welcome.html')

@app.route('/upload', methods=['POST','GET'])
def upload_file():

    if request.method == 'GET':
        return render_template('upload.html')
    else:
        file = request.files['filename']
        file.save(UPLOAD_FOLDER + file.filename)

        indices = {0: 'Cat', 1: 'Dog'}
        result = predict(UPLOAD_FOLDER + file.filename)

        #accuracy = round(result[0][predicted_class] * 100, 2)
        label = indices[result]
        print(label)

    return render_template('upload.html', filename = file.filename, label = result)

if __name__ == '__main__':
  app.run()